In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sets the theme of the charts
plt.style.use('seaborn-v0_8-darkgrid')

%matplotlib inline

In [2]:
# imports the csv files
appointments_df = pd.read_csv('appointments.csv', low_memory=False, encoding='unicode_escape')

In [3]:
# checks the content of the csv files
appointments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9752932 entries, 0 to 9752931
Data columns (total 11 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   pxid        object
 1   clinicid    object
 2   doctorid    object
 3   apptid      object
 4   status      object
 5   TimeQueued  object
 6   QueueDate   object
 7   StartTime   object
 8   EndTime     object
 9   type        object
 10  Virtual     object
dtypes: object(11)
memory usage: 818.5+ MB


### `pxid` variable

In [4]:
# check values of pxid
appointments_df['pxid']

0          EF196B348A49FB32DABC9834DC4FAAD9
1          EAE3C87D0B33351272F2E9B9B1B56217
2          7C5C93809D626CC702D08F33985B2B58
3          C300C2B9E0E5D4C46E8093BCDBFA05CA
4          B3DBE7F9E4DC33CBC5660E0A923CF8E8
                         ...               
9752927    2583E761CF4CAB4813AAEAFDAA883CC6
9752928    F51E6BF96EA5028AE5F5C01EBF08E3BD
9752929    84E3EB4A060096C3702D33F5A52E8B43
9752930    2B9F701BED6F68800637ADB7EF4CACE2
9752931    2ED01D09EF929AE3CA7564A2CB09DC2C
Name: pxid, Length: 9752932, dtype: object

In [5]:
# check unique values of pxid
appointments_df['pxid'].unique()

array(['EF196B348A49FB32DABC9834DC4FAAD9',
       'EAE3C87D0B33351272F2E9B9B1B56217',
       '7C5C93809D626CC702D08F33985B2B58', ...,
       '84E3EB4A060096C3702D33F5A52E8B43',
       '2B9F701BED6F68800637ADB7EF4CACE2',
       '2ED01D09EF929AE3CA7564A2CB09DC2C'], dtype=object)

In [6]:
# check for null values of pxid
pxid_count = appointments_df['pxid'].isnull().sum()

### `clinicid` variable

In [7]:
# check values of clinicid
appointments_df['clinicid']

0          ADF7EE2DCF142B0E11888E72B43FCB75
1          1E0F65EB20ACBFB27EE05DDC000B50EC
2          1E0F65EB20ACBFB27EE05DDC000B50EC
3          98C39996BF1543E974747A2549B3107C
4          77EE3BC58CE560B86C2B59363281E914
                         ...               
9752927    CCFC2D538DDFF519D893A6B966A1C4F1
9752928    CCFC2D538DDFF519D893A6B966A1C4F1
9752929    CCFC2D538DDFF519D893A6B966A1C4F1
9752930    CCFC2D538DDFF519D893A6B966A1C4F1
9752931    40F4775D64533EE66E3E20AE64228661
Name: clinicid, Length: 9752932, dtype: object

In [8]:
# check unique values of clinicid
appointments_df['clinicid'].unique()

array(['ADF7EE2DCF142B0E11888E72B43FCB75',
       '1E0F65EB20ACBFB27EE05DDC000B50EC',
       '98C39996BF1543E974747A2549B3107C', ...,
       '772C1C73D4FFE88046A3CDB8772E53EB',
       'CCFC2D538DDFF519D893A6B966A1C4F1',
       '40F4775D64533EE66E3E20AE64228661'], dtype=object)

In [9]:
# check for null values of clinicid
appointments_df['clinicid'].isnull().sum()

0

### `doctorid` variable

In [10]:
# check values of doctorid
appointments_df['doctorid']

0          BB04AF0F7ECAEE4AAE62035497DA1387
1          82AA4B0AF34C2313A562076992E50AA3
2          82AA4B0AF34C2313A562076992E50AA3
3          AD61AB143223EFBC24C7D2583BE69251
4          AD61AB143223EFBC24C7D2583BE69251
                         ...               
9752927    B706835DE79A2B4E80506F582AF3676A
9752928    B706835DE79A2B4E80506F582AF3676A
9752929    B706835DE79A2B4E80506F582AF3676A
9752930    B706835DE79A2B4E80506F582AF3676A
9752931    B706835DE79A2B4E80506F582AF3676A
Name: doctorid, Length: 9752932, dtype: object

In [11]:
# check unique values of doctorid
appointments_df['doctorid'].unique()

array(['BB04AF0F7ECAEE4AAE62035497DA1387',
       '82AA4B0AF34C2313A562076992E50AA3',
       'AD61AB143223EFBC24C7D2583BE69251', ...,
       '69B4FA3BE19BDF400DF34E41B93636A4',
       '4FC8ED929E539525E3590F1607718F97',
       'B706835DE79A2B4E80506F582AF3676A'], dtype=object)

In [12]:
# check unique values of doctorid
appointments_df['doctorid'].isnull().sum()

0

### `apptid` variable

In [13]:
# check values of apptid
appointments_df['apptid']

0          C1CC0949B93D00A559F7A0BD38361E80
1          6585A31C60A1886FBA1433C50012B504
2          7250DCFF615E6580295C7E6ED4322371
3          F5BBDCC08E39332F0AC27BB95CF1396A
4          55783FB42A02DABC3B2ED239B924DC87
                         ...               
9752927    215A26E2E5B831D87C41A5C1CC382B1C
9752928    E9EB60E3DA311FAE308A2DB53B6CCBAC
9752929    EFE2DDFD855E2B4E1C67956CE1CE8A6E
9752930    434410208F17F50143E1CE66E4CEF23F
9752931    62D462EFFD157E70C4301E555962D44B
Name: apptid, Length: 9752932, dtype: object

In [14]:
# check unique values of apptid
appointments_df['apptid'].unique()

array(['C1CC0949B93D00A559F7A0BD38361E80',
       '6585A31C60A1886FBA1433C50012B504',
       '7250DCFF615E6580295C7E6ED4322371', ...,
       'EFE2DDFD855E2B4E1C67956CE1CE8A6E',
       '434410208F17F50143E1CE66E4CEF23F',
       '62D462EFFD157E70C4301E555962D44B'], dtype=object)

In [15]:
# check unique values of apptid
appointments_df['apptid'].isnull().sum()

0

### `status` variable

In [16]:
# check values of status
appointments_df['status']

0          Complete
1            Queued
2            Queued
3            Queued
4            Queued
             ...   
9752927      Queued
9752928      Queued
9752929      Queued
9752930      Queued
9752931      Queued
Name: status, Length: 9752932, dtype: object

In [17]:
# check unique values of status
appointments_df['status'].unique()

array(['Complete', 'Queued', 'NoShow', 'Serving', 'Cancel', 'Skip',
       'Completed', 'Admitted'], dtype=object)

In [18]:
# check unique values of status
appointments_df['status'].isnull().sum()

0

### `TimeQueued` variable

In [19]:
# check values of TimeQueued
appointments_df['TimeQueued']

0                    2018-04-11 01:59:58
1                    2018-04-10 10:34:16
2                    2018-04-03 15:26:19
3          2015-11-24 08:01:13.920000000
4          2015-11-24 08:01:00.420000000
                       ...              
9752927    2023-11-20 00:34:25.337000000
9752928    2023-12-05 07:51:25.443000000
9752929    2024-01-04 06:11:06.437000000
9752930    2024-01-13 06:12:54.737000000
9752931    2024-01-04 00:45:05.063000000
Name: TimeQueued, Length: 9752932, dtype: object

In [20]:
# check unique values of TimeQueued
appointments_df['TimeQueued'].unique()

array(['2018-04-11 01:59:58', '2018-04-10 10:34:16',
       '2018-04-03 15:26:19', ..., '2024-01-04 06:11:06.437000000',
       '2024-01-13 06:12:54.737000000', '2024-01-04 00:45:05.063000000'],
      dtype=object)

In [21]:
# check unique values of TimeQueued
appointments_df['TimeQueued'].isnull().sum()

0

### `QueueDate` variable

In [22]:
# check values of QueueDate
appointments_df['QueueDate']

0          2018-04-10 16:00:00
1          2018-04-08 16:00:00
2          2018-03-30 16:00:00
3                          NaN
4                          NaN
                  ...         
9752927    2023-11-22 16:00:00
9752928    2023-12-04 16:00:00
9752929    2024-01-03 16:00:00
9752930    2024-01-12 16:00:00
9752931    2024-01-03 16:00:00
Name: QueueDate, Length: 9752932, dtype: object

In [23]:
# check unique values of QueueDate
appointments_df['QueueDate'].unique()

array(['2018-04-10 16:00:00', '2018-04-08 16:00:00',
       '2018-03-30 16:00:00', ..., '2026-01-22 16:00:00',
       '2008-12-30 16:00:00', '2026-07-14 16:00:00'], dtype=object)

In [24]:
# check unique values of QueueDate
appointments_df['QueueDate'].isnull().sum()

18

### `StartTime` variable

In [25]:
# check values of StartTime
appointments_df['StartTime']

0                          NaN
1          2018-04-09 10:33:00
2          2018-03-31 15:25:00
3                          NaN
4                          NaN
                  ...         
9752927                    NaN
9752928                    NaN
9752929    2024-01-04 08:00:00
9752930    2024-01-13 08:00:00
9752931    2024-01-04 02:30:00
Name: StartTime, Length: 9752932, dtype: object

In [26]:
# check unique values of StartTime
appointments_df['StartTime'].unique()

array([nan, '2018-04-09 10:33:00', '2018-03-31 15:25:00', ...,
       '2023-07-20 07:30:33', '2023-08-08 08:00:55',
       '2023-11-03 19:30:00'], dtype=object)

In [27]:
# check unique values of StartTime
appointments_df['StartTime'].isnull().sum()

5879400

### `EndTime` variable

In [28]:
# check values of EndTime
appointments_df['EndTime']

0                          NaN
1          2018-04-12 05:29:57
2          2018-04-05 15:51:47
3                          NaN
4                          NaN
                  ...         
9752927                    NaN
9752928                    NaN
9752929    2024-01-04 08:15:00
9752930    2024-01-13 08:15:00
9752931    2024-01-04 02:45:00
Name: EndTime, Length: 9752932, dtype: object

In [29]:
# check unique values of EndTime
appointments_df['EndTime'].unique()

array([nan, '2018-04-12 05:29:57', '2018-04-05 15:51:47', ...,
       '2023-07-20 07:45:33', '2023-08-08 08:15:55',
       '2023-11-03 19:45:00'], dtype=object)

In [30]:
# check unique values of EndTime
appointments_df['EndTime'].isnull().sum()

6036781

### `type` variable

In [31]:
# check values of type
appointments_df['type']

0          Consultation
1             Inpatient
2             Inpatient
3          Consultation
4          Consultation
               ...     
9752927    Consultation
9752928    Consultation
9752929    Consultation
9752930    Consultation
9752931    Consultation
Name: type, Length: 9752932, dtype: object

In [32]:
# check unique values of type
appointments_df['type'].unique()

array(['Consultation', 'Inpatient'], dtype=object)

In [33]:
# check unique values of type
appointments_df['type'].isnull().sum()

0

### `Virtual` variable

In [34]:
# check values of Virtual
appointments_df['Virtual']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
9752927    False
9752928    False
9752929    False
9752930    False
9752931    False
Name: Virtual, Length: 9752932, dtype: object

In [35]:
# check unique values of Virtual
appointments_df['Virtual'].unique()

array([nan, False, True], dtype=object)

In [36]:
# check unique values of Virtual
appointments_df['Virtual'].isnull().sum()

1952593